# Does urban vulnerability form spatio-temporal patterns? 

In [ ]:
%load_ext autoreload
%autoreload 2

In [34]:
import pandas as pd
import geopandas as gpd 
from matplotlib import pyplot as plt
import matplotlib
import numpy as np
from scipy.spatial.distance import euclidean
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from mycolorpy import colorlist as mcp
from fastdtw import fastdtw
from ema_workbench.analysis import clusterer

from src.ts_clusterer import prepare_city_features, arrange_cluster_labels
from src.visualizer import vis_cluster_lineplots, vis_clusters_maps

In [35]:
scaled_data = pd.read_csv(f"../data/processed/p2000/scaled_spatio_temporal_grid_time_step=4.csv", index_col=0)
borders = gpd.read_file("../data/processed/cbs/wijk_buurt_kaart/cities.json")
grid = gpd.read_file("../data/processed/cbs/kaart_met_statistieken/1000x1000.json")

## Clustering scaled calls with agglomerative clustering (round 1)

In [ ]:
# Do clustering
n_clusters = 16
metric = "euclidean" # euclidean
linkage = "ward" # ward
data = scaled_data.values
print('N clusters: ', n_clusters)
print('Selected linkage: ', linkage)
print('Selected metric: ', metric)

clusters = clusterer.apply_agglomerative_clustering(data, n_clusters, metric=metric, linkage=linkage)

# Assign labels
data = scaled_data.copy()
data['label'] = clusters

# Re-arrange labels to be in order of increasing count
data = arrange_cluster_labels(data)
clusters = data['label'].values
cluster_labels = data['label'].value_counts().index.values

## Analyzing metrics

In [ ]:
silhouette_scores = {}
calinski_harabaz_scores = {}
daviess_bouldin_scores = {}

for i in range(2, 30):
    clusters = clusterer.apply_agglomerative_clustering(data, i, metric=metric, linkage=linkage)
    silhouette_scores[i] = silhouette_score(data, clusters, metric=metric)
    calinski_harabaz_scores[i] = calinski_harabasz_score(data, clusters)
    daviess_bouldin_scores[i] = davies_bouldin_score(data, clusters)

all_metrics = pd.DataFrame([silhouette_scores, calinski_harabaz_scores, daviess_bouldin_scores]).T
all_metrics.columns = ['Silhouette Score', 'Calinski-Harabasz', 'Davies-Bouldin']
all_metrics.plot(subplots=True, figsize=(4, 7), layout=(3, 1), title=['Silhouette Score', 'Calinski-Harabasz', 'Davies-Bouldin'], legend=False)
for ax in plt.gcf().axes:
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
for ax in plt.gcf().axes:
    ax.xaxis.set_major_locator(plt.MaxNLocator(10))
# plt.savefig('../report/figures/appendix/sfig5.png', dpi=300, bbox_inches='tight')
# all_metrics.to_csv('../report/tables/appendix/stab5.csv')

In [38]:
all_metrics.index.name = 'N clusters'
all_metrics.style.background_gradient(cmap='Blues', subset=['Silhouette Score'])\
                 .background_gradient(cmap='Greens', subset=['Calinski-Harabasz'])\
                 .background_gradient(cmap='Reds', subset=['Davies-Bouldin'])\
                 .format("{:.2f}", subset=['Silhouette Score', 'Davies-Bouldin'])\
                 .format("{:.0f}", subset=['Calinski-Harabasz'])\
                 .to_latex(buf='../report/tables/appendix/stab5.tex',
                        caption='.',
                        position='H',
                        convert_css=True,
                        hrules=True,
                        label='tab:cluster_metrics',
                        sparse_index=True,
                        sparse_columns=True,
                        multicol_align="|c|",
                        position_float="centering",
                        )

## Improving clustering results by manually analyzing the distance matrix (round 2) 

In [ ]:
k_min = 6
k_max = 16
df = data[data['label'].isin(np.arange(k_min, k_max + 1))].copy()

# Calculate mean of each cluster
means = data[data['label'].isin(np.arange(k_min))].groupby('label').mean()

df['label'] = np.nan
df['dist'] = np.nan

# Calculate distance to each mean
for i, row in df.iterrows():
    min_dist = np.inf
    label = np.nan
    for j, mean in means.iterrows():
        # Get euclidean distance
        dist = euclidean(row[:-2], mean)
        # dist = fastdtw(row[:-2].values.reshape(-1,1), mean.values.reshape(-1,1), dist=euclidean)[0]
        if dist < min_dist:
            min_dist = dist
            label = j
    df.loc[i, 'label'] = label
    df.loc[i, 'dist'] = min_dist
df['label'] = df['label'].astype(int)

# Select only points that are close to the cluster
dist_threshold = 1.25

df = df[df['dist'] < dist_threshold]
df

In [40]:
top_n = data[data['label'].isin(cluster_labels[:k_min])].copy()
y = pd.concat([top_n, df.drop(columns=['dist'])], axis=0)

## Plotting

In [41]:
my_colors = ['#FC2E20',
             '#FD7F20',
             '#1F77B4',
             '#AEC7E8',
             '#A89F91',
             '#7E735F']

In [ ]:
# Plot the new clusters
vis_cluster_lineplots(y, k_min, cluster_labels[:k_min], random_colors=False, my_colors=my_colors)
# plt.savefig('../report/figures/fig3.png', bbox_inches='tight', dpi=300)

In [ ]:
vis_clusters_maps(y, grid, borders, k_min, k_max=k_min, random_colors=False, my_colors=my_colors, save_fig=False)

In [32]:
y = pd.merge(grid[['c28992r1000', 'geometry']], y, left_on='c28992r1000', right_index=True, how='right')
y.to_file('../results/cluster_labels.json', driver="GeoJSON")